In [1]:
import hash_blocking_spark as hash
import ngram_blocking_spark as ngram
import matchers_spark as match
import similarity_spark as sim
import blocking_structured_and_sorted_spark as ss
import cluster_spark as cluster

In [2]:
from pyspark.sql import SparkSession
import pandas as pd

dblp_csv = '../CSV-files/dblp_stem.csv'
dblp = pd.read_csv(dblp_csv)

acm_csv = '../CSV-files/acm_stem.csv'
acm = pd.read_csv(acm_csv)

selected_columns = ['paper_title']
spark = SparkSession.builder.appName("sparkk").getOrCreate()
acm_df = spark.read.csv(acm_csv, header=True, inferSchema=True)
dblp_df = spark.read.csv(dblp_csv, header=True, inferSchema=True)


# this was in our experiment in two the best match
a = hash.initial_hash_parallel_df(spark,acm_df, selected_columns)
b = hash.initial_hash_parallel_df(spark, dblp_df, selected_columns)
c = match.apply_similarity_blocks_spark(a,b, 0.7, sim.jaccard_similarity)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/30 15:42:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Processing time: 2.375669002532959 seconds. Number of index combinations: 1052


In [3]:
import csv

def write_index_pairs_to_csv(data, filename):
    header = ['dblp_index', 'acm_index']
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)  # Write the header
        for row in data:
            cleaned_row = [row[0].strip('[]'), row[1].strip('[]')]  # Remove brackets
            writer.writerow(cleaned_row)

write_index_pairs_to_csv(c, '../Matched/Matched_spark.csv')
 

In [4]:
# get matched entites as a list back for spark and without
import pandas as pd
def reconstructed_pairs(path):
    df_pairs = pd.read_csv(path)
    return list(zip(df_pairs['dblp_index'], df_pairs['acm_index']))

pairs = reconstructed_pairs('../Matched/Matched Entities.csv')
pairs_spark = reconstructed_pairs('../Matched/Matched_spark.csv')

In [13]:
# check if they are equal
def pairs_correspond(pair_list1, pair_list2):
    sorted_pair_list1 = [tuple(sorted(pair)) for pair in pair_list1]
    sorted_pair_list2 = [tuple(sorted(pair)) for pair in pair_list2]

    for pair1 in sorted_pair_list1:
        if pair1 not in sorted_pair_list2:
            return False
    return True

pairs_correspond(pairs_spark, pairs)

True

In [14]:
import csv
# cluster the spark pairs which are equals
def cluster_to_csv(cluster_data, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for idx, item in enumerate(cluster_data, start=1):
            writer.writerow([f'Cluster {idx}: {item}'])

cluster_pairs = cluster.build_clusters_parallel(pairs_spark)
cluster_to_csv(cluster_pairs, '/Users/aliaslan/Documents/GitHub/DIA-ER/Matched/Spark_Cluster.csv')
print(len(cluster_pairs))

24/01/30 15:48:56 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


959


In [ ]:
from pyspark.sql import SparkSession

dblp_csv = '../CSV-files/dblp_stem.csv'
dblp = pd.read_csv(dblp_csv)

acm_csv = '../CSV-files/acm_stem.csv'
acm = pd.read_csv(acm_csv)
selected_columns = ['paper_title']


for i in range(3):
    spark = SparkSession.builder.appName("sparkk").getOrCreate()
    acm_df = spark.read.csv(acm_csv, header=True, inferSchema=True)
    dblp_df = spark.read.csv(dblp_csv, header=True, inferSchema=True)
    
    c = hash.initial_hash_parallel_df(spark,acm_df, selected_columns)
    d = hash.initial_hash_parallel_df(spark, dblp_df, selected_columns)
    e = match.apply_similarity_blocks_spark(c,d, 0.7, sim.jaccard_similarity)
    spark.stop

Processing time: 2.2450268268585205 seconds. Number of index combinations: 1052


<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x11b3e8310>>

In [6]:
# Example that other are functioning

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkk").getOrCreate()

acm_df = spark.read.csv(acm_csv, header=True, inferSchema=True)
dblp_df = spark.read.csv(dblp_csv, header=True, inferSchema=True)

selected_columns = ['author_names', 'paper_title']
c = ngram.initial_ngram_parallel_df(spark,acm_df, selected_columns,2)
d = ngram.initial_ngram_parallel_df(spark, dblp_df, selected_columns,2)
e = match.apply_similarity_blocks_spark(c,d, 0.7, sim.jaccard_similarity)



ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

Processing time: 1.8916327953338623 seconds. Number of index combinations: 782


<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x11b3e8310>>